In [ ]:
%load_ext autoreload
%autoreload 2

%cd '..'

In [ ]:
import json

import networkx as nx
import matplotlib.pyplot as plt

from load.utils import load_network, load_subreddits
from network.utils import filter_node
from preprocessing.constants import METADATA_DIR, DATA_DIR


In [ ]:
YEARS = [year for year in range(2008, 2020)]


In [ ]:
subreddits = load_subreddits()

dem_subreddits = set(subreddits[subreddits["party"] == "dem"]["subreddit"])
rep_subreddits = set(subreddits[subreddits["party"] == "rep"]["subreddit"])


In [ ]:

blacklist = {"politics"}

eligible_subreddits = set(subreddits["subreddit"]) - blacklist

print(eligible_subreddits)

In [ ]:
print(dem_subreddits)

In [ ]:
print(rep_subreddits)

### Unweighted

In [ ]:
networks_unweighted = {year: load_network(year=year, weighted=False) for year in YEARS}


In [ ]:
dem_networks_unweighted = {}
rep_networks_unweighted = {}

dem_subreddits_unweighted = {}
rep_subreddits_unweighted = {}

for year, network_unweighted in networks_unweighted.items():
    print(year)
    active_subreddits = set(network_unweighted.nodes())
    dem_active_subreddits = (dem_subreddits & active_subreddits)
    rep_active_subreddits = (rep_subreddits & active_subreddits)
    
    print("dem seed", list(dem_active_subreddits))
    print("rep seed", list(rep_active_subreddits))
    # Democrats

    dem_network_unweighted = nx.subgraph_view(  # type: ignore
        network_unweighted,
        filter_node=lambda node: filter_node(
            node,
            network=network_unweighted,
            eligible_subreddits=eligible_subreddits,
            party_subreddits=dem_active_subreddits,
            opposition_subreddits=rep_active_subreddits,
            weighted=False,
        ),
    )
    dem_networks_unweighted[year] = dem_network_unweighted

    dem_subreddits_unweighted[year] = list(set(dem_network_unweighted.nodes()))


    # Draw democrat network
    A_dem = nx.nx_agraph.to_agraph(dem_network_unweighted)
    A_dem.node_attr["style"] = "filled"

    for dem_active_subreddit in dem_active_subreddits:
        n = A_dem.get_node(dem_active_subreddit)
        n.attr["fillcolor"] = "#00AEF3"

    A_dem.layout(prog="dot")  # use dot
    
    A_dem.write(f"{DATA_DIR}/figures/partisan_networks/dem_network_{year}_unweighted.pdf")
    A_dem.draw(f"{DATA_DIR}/figures/partisan_networks/dem_network_{year}_unweighted.pdf")

    # Republicans

    rep_network_unweighted = nx.subgraph_view(  # type: ignore
        network_unweighted,
        filter_node=lambda node: filter_node(
            node,
            network=network_unweighted,
            eligible_subreddits=eligible_subreddits,
            party_subreddits=rep_active_subreddits,
            opposition_subreddits=dem_active_subreddits,
            weighted=False,
        ),
    )

    rep_networks_unweighted[year] = rep_network_unweighted
    
    rep_subreddits_unweighted[year] = list(set(rep_network_unweighted.nodes()))

    print(f"Dem subreddits {year}: seed -> {len(dem_active_subreddits)}, final -> {len(dem_subreddits_unweighted[year])}")
    print(f"Rep subreddits {year}: seed -> {len(dem_active_subreddits)}, final -> {len(rep_subreddits_unweighted[year])}")

    print("Discovered dem subreddits", list(set(dem_network_unweighted.nodes()) - dem_active_subreddits))
    print("Discovered rep subreddits", list(set(rep_network_unweighted.nodes()) - rep_active_subreddits))

    # Draw republican network

    plt.figure(3,figsize=(11.69,8.27)) 

    A_rep = nx.nx_agraph.to_agraph(rep_network_unweighted)
    A_rep.node_attr["style"] = "filled"

    for rep_active_subreddit in rep_active_subreddits:
        n = A_rep.get_node(rep_active_subreddit)
        n.attr["syle"] = "filled"
        n.attr["fillcolor"] = "#E81B23"
    
    A_rep.layout(prog="dot")  # use dot

    A_rep.write(f"{DATA_DIR}/figures/partisan_networks/rep_network_{year}_unweighted.pdf")
    A_rep.draw(f"{DATA_DIR}/figures/partisan_networks/rep_network_{year}_unweighted.pdf")

    plt.show()


In [ ]:
with open(
    f"{METADATA_DIR}/dem_subreddits_unweighted.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(dem_subreddits_unweighted, f)


In [ ]:
with open(
    f"{METADATA_DIR}/rep_subreddits_unweighted.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(rep_subreddits_unweighted, f)